In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/val-click-candidates-features/click_candidate_with_features_p2.pqt
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p3.pqt
/kaggle/input/val-click-candidates-features/val_buy.pqt
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p9.pqt
/kaggle/input/val-click-candidates-features/__results__.html
/kaggle/input/val-click-candidates-features/user_features.pqt
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p8.pqt
/kaggle/input/val-click-candidates-features/val_click.pqt
/kaggle/input/val-click-candidates-features/__notebook__.ipynb
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p4.pqt
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p5.pqt
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p0.pqt
/kaggle/input/val-click-candidates-features/click_candidate_with_features_p7.pqt
/kaggle/input/val-click-candidates-features/click_ca

In [2]:
click_candidates = pd.read_parquet('/kaggle/input/val-click-candidates-features/click_candidates.pqt')

In [3]:
click_candidates['session'] = click_candidates['session'].astype('int32')

In [4]:
click_candidates

,session,aids,item_item_count,item_user_count,item_buy_ratio,user_user_count,user_item_count,user_buy_ratio,feature_click,feature_cart,feature_buy,click
0,11098528,11830,33776,19211,0.173970,1,1,0.0,1,0,0,0
1,11098528,588923,24588,14741,0.125346,1,1,0.0,0,0,0,0
2,11098528,1732105,10452,5770,0.280233,1,1,0.0,0,0,0,0
3,11098528,571762,18822,12405,0.124588,1,1,0.0,0,0,0,0
4,11098528,884502,31522,18163,0.135397,1,1,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
14999995,11198527,637827,12590,7183,0.140270,3,1,0.0,0,0,0,0
14999996,11198527,485256,36584,12763,0.214438,3,1,0.0,0,0,0,0
14999997,11198527,683401,6623,3234,0.116413,3,1,0.0,0,0,0,0
14999998,11198527,544144,37907,18053,0.238637,3,1,0.0,0,0,0,0


In [5]:
#click_candidates['session'][click_candidates['session']==11456333].value_counts()

In [6]:
click_candidates['session'][click_candidates['session']==11098529].value_counts()

11098529    150
Name: session, dtype: int64

In [7]:
import xgboost as xgb
from sklearn.model_selection import GroupKFold

skf = GroupKFold(n_splits=5)
for fold,(train_idx, valid_idx) in enumerate(skf.split(click_candidates, click_candidates['click'], groups=click_candidates['session'] )):

    X_train = click_candidates.iloc[train_idx, 2:-1]
    y_train = click_candidates.loc[train_idx, 'click']
    X_valid = click_candidates.iloc[valid_idx, 2:-1]
    y_valid = click_candidates.loc[valid_idx, 'click']

    # IF YOU HAVE 50 CANDIDATE WE USE 50 BELOW
    dtrain = xgb.DMatrix(X_train, y_train, group=[150] * (len(train_idx)//150) ) 
    dvalid = xgb.DMatrix(X_valid, y_valid, group=[150] * (len(valid_idx)//150) ) 

    xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
    model = xgb.train(xgb_parms, 
        dtrain=dtrain,
        evals=[(dtrain,'train'),(dvalid,'valid')],
        num_boost_round=1000,
        verbose_eval=100)
    model.save_model(f'XGB_fold{fold}_click.xgb')

[0]	train-map:0.65896	valid-map:0.66740
[100]	train-map:0.64317	valid-map:0.64520
[200]	train-map:0.64654	valid-map:0.64493
[300]	train-map:0.65039	valid-map:0.64463
[400]	train-map:0.65395	valid-map:0.64440
[500]	train-map:0.65818	valid-map:0.64328
[600]	train-map:0.66084	valid-map:0.64304
[700]	train-map:0.66369	valid-map:0.64279
[800]	train-map:0.66637	valid-map:0.64276
[900]	train-map:0.66874	valid-map:0.64252
[999]	train-map:0.67102	valid-map:0.64215
[0]	train-map:0.66169	valid-map:0.65749
[100]	train-map:0.64500	valid-map:0.63453
[200]	train-map:0.64879	valid-map:0.63533
[300]	train-map:0.65255	valid-map:0.63525
[400]	train-map:0.65615	valid-map:0.63571
[500]	train-map:0.65966	valid-map:0.63554
[600]	train-map:0.66253	valid-map:0.63544
[700]	train-map:0.66528	valid-map:0.63543
[800]	train-map:0.66752	valid-map:0.63498
[900]	train-map:0.66971	valid-map:0.63443
[999]	train-map:0.67204	valid-map:0.63422
[0]	train-map:0.66137	valid-map:0.65806
[100]	train-map:0.64437	valid-map:0.6363